In [78]:
%reload_ext autoreload
%autoreload 2


from bartez.boards import *
#get_board_preset_small_01 
blocks, geometry = generate_board_from_string(get_board_preset_01())

In [79]:
from bartez.crossword import Crossworld

crossword = Crossworld(geometry[0], geometry[1])
crossword.set_blocks(blocks)
crossword.prepare()
#crossword.print_crossword()
#crossword.print_entries()

In [80]:
import networkx as nx

G_entries = nx.Graph()
entries = crossword.get_entries()
nodes_h = []
nodes_v = []
for i, e in enumerate(entries):
    #print(e.get_description() + ", index: " + str(e.absolute_index()))
    relations = e.get_relations()
    is_horizontal = e.is_horizontal()
    e_index = e.get_absolute_index()
    for rel in relations:
        G_entries.add_edge(e_index, rel.get_index())
    
    G_entries.nodes[i]['id']=e_index
    G_entries.nodes[i]['label']=e.get_description()
    G_entries.nodes[i]['horizontal']=is_horizontal
    G_entries.nodes[i]['color']= "00ff1e" if is_horizontal else "0bcde"
    
    if is_horizontal:
        nodes_h.append(e_index)
    else:
        nodes_v.append(e_index)

#print(G.nodes())
#print(G.edges())

In [81]:
%matplotlib notebook

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt


In [82]:
from pyvis.network import Network
import networkx as nx

nt = Network('600px', '800px', notebook=True)
# populates the nodes and edges data structures
nt.from_nx(G_entries)
#nt.show('nx.html')#import igraph as ig


#IG = ig.Graph()
#IG.from_networkx(G)
#print(IG)
#crossword.print_crossword()

In [83]:
def create_graph_nephews_by_orientation(entries, orientation_h):
    # will extract either horizontal or vertical nodes,
    # and will collpase 1 level of edges to connect the nodes
    # with the same orientation
    G = nx.Graph()
    
    for i, e in enumerate(entries):
        is_horizontal = e.is_horizontal()
        
        if orientation_h != is_horizontal:
            continue

        relations = e.get_relations()
        e_index = e.get_absolute_index()
        
        for rel in relations:
            #G.add_edge(e_index, rel.get_index())
            relation_index = rel.get_index()
            relation_entry = entries[relation_index]
            relations_with_same_orientation = relation_entry.get_relations()
            
            for rel_wso in relations_with_same_orientation:
                rel_wso_index = rel_wso.get_index()
                if rel_wso_index == e_index:
                    continue
                    
                rel_wso_entry = entries[rel_wso_index]
                assert(orientation_h == rel_wso_entry.is_horizontal())
                G.add_edge(e_index, rel_wso_index)
        G.nodes[i]['id']=e_index
        G.nodes[i]['title']=e.get_description()
        G_entries.nodes[i]['color']= "00ff1e" if is_horizontal else "0bcde"
        
    return G

In [84]:
def create_graph_collapsed_by_orientation(entries, orientation_h):
    # will extract either horizontal or vertical nodes,
    # and will collpase 1 level of edges to connect the nodes
    # with the same orientation
    G = nx.Graph()
    
    for i, e in enumerate(entries):
        is_horizontal = e.is_horizontal()
        
        if orientation_h != is_horizontal:
            continue

        e_index = e.get_absolute_index()
        e_start = e.get_coordinate()
        e_end = e.get_coordinate_end()
        e_desc = e.get_description()
        
        for j, o in enumerate(entries):
            o_index = o.get_absolute_index()
            o_entry = entries[o_index]
            o_start = o_entry.get_coordinate()
            o_end = o_entry.get_coordinate_end()
            o_desc = o_entry.get_description()
            
            if orientation_h != o_entry.is_horizontal():
                continue

            condition_h =  is_horizontal
            condition_h &= (o_start.x() in [e_start.x() - 1, e_start.x() + 1]) or \
                           (o_end.x()   in [e_start.x() - 1, e_start.x() + 1])
            condition_h &= ((o_start.y() >= e_start.y()) and (o_start.y() <= e_end.y()) or \
                            (o_end.y()   >= e_start.y()) and (o_end.y()   <= e_end.y()))
            
            condition_v =  is_horizontal == False
            condition_v &= (o_start.y() in [e_start.y() - 1, e_start.y() + 1]) or \
                           (o_end.y()   in [e_start.y() - 1, e_start.y() + 1])
            condition_v &= ((o_start.x() >= e_start.x()) and (o_start.x() <= e_end.x()) or \
                            (o_end.x()   >= e_start.x()) and (o_end.x()   <= e_end.x()))
            
            if (orientation_h and condition_h) or (orientation_h == False and condition_v):
                G.add_edge(e_index, o_index)
                #G.nodes[e_index]['id']=e_index
                G.nodes[e_index]['id']=e.get_description()
                G.nodes[e_index]['title']=e.get_description()
                G.nodes[e_index]['color']= "00ff1e" if is_horizontal else "0bcde"
                G.nodes[o_index]['id']=o_index
                G.nodes[o_index]['title']=o_entry.get_description()
                G.nodes[o_index]['color']= "00ff1e" if is_horizontal else "0bcde"
                assert(orientation_h == o_entry.is_horizontal())                
                #print("**************************")
                #print("E: " + e_desc)
                #print("  P: " + o_desc)
                #print("")
        
    return G

In [85]:
GH = create_graph_collapsed_by_orientation(entries, True)
GV = create_graph_collapsed_by_orientation(entries, False)


In [86]:
nt_h = Network('400px', '800px', notebook=True)
# populates the nodes and edges data structures
nt_h.from_nx(GH)
#nt_h.show('nt\e_collapsed_h.html')#import igraph as ig

#crossword.print_crossword()

In [87]:
nt_h = Network('400px', '800px', notebook=True)
# populates the nodes and edges data structures
nt_h.from_nx(GV)
#1nt_h.show('nt\e_collapsed_v.html')#import igraph as ig

In [88]:
crossword.print_crossword()



. . . . # . . . . . . . # . . . . . 
. . . . . . . . . # . # . . . . . . 
. . # . . . . . # . . . . . . . # . 
. # . . . . . # . . . . . . . # . . 
# . . . . . # . . . . . . . # . . . 
. . . . . # . . . . . . . # . . . . 
. . . . # . . . . . . . # . . . . . 
. . . # . . . . . . . # . . . . . # 
. . # . . . . . . . # . . . . . # . 
. # . . . . . . . # . . . . . # . . 
. . . . . . # . # . . . . . . . . . 
. . . . . # . . . . . . . # . . . . 



In [89]:
#nx.algorithms.cluster.clustering(G_entries)
#nx.algorithms.cluster.generalized_degree(G_entries)
#nx.algorithms.communicability_alg.communicability(G_entries)
eigenvector_centrality = nx.algorithms.centrality.eigenvector_centrality_numpy(G_entries)
eigenvector_centrality_sorted = {k: v for k, v in sorted(eigenvector_centrality.items(), key=lambda item: item[1])}
for index, centrality in eigenvector_centrality_sorted.items():
    #print(str(index) + ":" + entries[index].get_description() + " - " + str(centrality) )
    continue

In [90]:
%reload_ext autoreload
%autoreload 2

from copy import copy, deepcopy

#crossword.print_entries()
entries_len = len(entries)

def add_column_to_matrix(matrix, column):
    new_matrix = []
    
    for col in column:
        for row in matrix:
            new_row = [r for r in row]
            new_row.append(col)
            #print(new_row)
            new_matrix.append(new_row)
    return new_matrix
            
list1 = ['a', 'b']
list11 = add_column_to_matrix([list1] ,list1)
list111 = add_column_to_matrix(list11 ,list1)
#print(list1)
#print(list11)
#print(list111)

indices1 = [  x.absolute_index()  for x in entries]
indices2 = [ [x.absolute_index()] for x in entries]
indices11 = add_column_to_matrix(indices2 ,indices1)
indices111 = add_column_to_matrix(indices11 ,indices1)

#print(indices1)
#print(indices11)
#print(len(indices111))
#print(indices111[-100:]


#GR = copy(G_entries)
for nodes_to_remove in indices11:
    GR = deepcopy(G_entries)
    #print(GR)
    for node in nodes_to_remove:
        if GR.has_node(node) == False:
            continue
            
        GR.remove_node(node)
        
    if nx.algorithms.components.is_connected(GR):
        continue
    
    connected_components_count = nx.algorithms.components.number_connected_components(GR)
    #print("nodes count: " + str(len(GR.nodes())))
    #print("Removing " + str(nodes_to_remove) + " makes GR not connected")
    
    #for node in nodes_to_remove:
    #    print( " + node: " + entries[node].get_description() )

    #print(" + Connected components: " + str(connected_components_count))          

In [91]:
#list(nx.algorithms.clique.find_cliques(G_entries))
list(nx.algorithms.community.kernighan_lin.kernighan_lin_bisection(G_entries))

[{0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  16,
  17,
  21,
  22,
  26,
  30,
  31,
  36,
  37,
  42,
  43,
  47,
  48,
  52,
  53,
  57,
  58,
  59,
  62,
  63,
  64,
  65,
  68,
  69,
  70,
  72,
  73},
 {10,
  11,
  12,
  13,
  14,
  15,
  18,
  19,
  20,
  23,
  24,
  25,
  27,
  28,
  29,
  32,
  33,
  34,
  35,
  38,
  39,
  40,
  41,
  44,
  45,
  46,
  49,
  50,
  51,
  54,
  55,
  56,
  60,
  61,
  66,
  67,
  71}]